In [153]:
#using the-odds-api https://the-odds-api.com/liveapi/guides/v4/#overview
import requests
import pandas as pd
import json
from typing import Union

In [164]:
url = 'https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds'

API_KEY = '3fb30eb65dfb05f1ac41a22af776598e'
REGIONS = 'us'
ODDS_FORMAT = 'decimal'
MARKETS = 'h2h,spreads'
DATE_FORMAT = 'iso'

STAKE: float = 500.0

In [4]:
api_response = requests.get(url, params={
    'apiKey': API_KEY,
    'regions': REGIONS,
    'oddsFormat': ODDS_FORMAT,
    'dateFormat': DATE_FORMAT
})

In [5]:
if api_response.status_code != 200:
    print(f'Failed to get odds: status_code {api_response.status_code}, response body {api_response.text}')

else:
    odds_json = api_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', api_response.headers['x-requests-remaining'])
    print('Used requests', api_response.headers['x-requests-used'])

Number of events: 27
[{'id': 'e8dce84fd844264b94713f2ce384fbcb', 'sport_key': 'americanfootball_nfl', 'sport_title': 'NFL', 'commence_time': '2022-11-13T14:30:00Z', 'home_team': 'Tampa Bay Buccaneers', 'away_team': 'Seattle Seahawks', 'bookmakers': [{'key': 'fanduel', 'title': 'FanDuel', 'last_update': '2022-11-12T07:56:33Z', 'markets': [{'key': 'h2h', 'outcomes': [{'name': 'Seattle Seahawks', 'price': 2.2}, {'name': 'Tampa Bay Buccaneers', 'price': 1.7}]}]}, {'key': 'betonlineag', 'title': 'BetOnline.ag', 'last_update': '2022-11-12T08:00:37Z', 'markets': [{'key': 'h2h', 'outcomes': [{'name': 'Seattle Seahawks', 'price': 2.27}, {'name': 'Tampa Bay Buccaneers', 'price': 1.68}]}]}, {'key': 'twinspires', 'title': 'TwinSpires', 'last_update': '2022-11-12T07:57:38Z', 'markets': [{'key': 'h2h', 'outcomes': [{'name': 'Seattle Seahawks', 'price': 2.23}, {'name': 'Tampa Bay Buccaneers', 'price': 1.68}]}]}, {'key': 'sugarhouse', 'title': 'SugarHouse', 'last_update': '2022-11-12T08:00:15Z', 'mark

In [6]:
df = pd.DataFrame(odds_json)
print(df)

                                  id             sport_key sport_title  \
0   e8dce84fd844264b94713f2ce384fbcb  americanfootball_nfl         NFL   
1   97f9ebd005616e0287299b00e546a819  americanfootball_nfl         NFL   
2   a62c2d9f2331d1db1d4acd8a07e05caa  americanfootball_nfl         NFL   
3   d2e4c7e39c1fe3cbb4f933c8b2d0fbf2  americanfootball_nfl         NFL   
4   92490f29e85c8e40206640dedf667441  americanfootball_nfl         NFL   
5   7b024b3be1e733850d3297ccb2311aac  americanfootball_nfl         NFL   
6   0148d027b7cb8dc406855ea91a6b7978  americanfootball_nfl         NFL   
7   27c35e3d77cd2d04f67575352f6aea37  americanfootball_nfl         NFL   
8   73ff0a347489c3e14ef1edf087e5262c  americanfootball_nfl         NFL   
9   dc14776bbc570877bb13425bd8c4db0a  americanfootball_nfl         NFL   
10  49b82cfa8d543643cf2b5d5109afa7be  americanfootball_nfl         NFL   
11  04b8e0aa7595d6a8497456b3144ae859  americanfootball_nfl         NFL   
12  e4cb60c1cd96813bbf67450007cb2a10  

In [10]:
# df.to_excel("odds.xlsx")

In [151]:
class odds:
    def __init__(self, bookie, t1, t2, price1, price2):
        self.bookie = bookie
        self.team1 = t1
        self.team2 = t2
        self.price1 = price1
        self.price2 = price2

class Game:
    def __init__(self, id, t1, t2):
        self.gameid = id
        self.t1 = t1
        self.t2 = t2
        self.odds = []
    def addodd(self, book, lastupdated, price1, price2):
        self.odds.append([book, lastupdated, price1, price2])
    def getodd(self, bookmaker: str)-> Union[list, None]:
        for odd in self.odds:
            if odd[0] == bookmaker:
                return [odd[2],odd[3]]
        return None
    def __str__(self):
        temp = "Game: " + self.t1 + " vs. " + self.t2 + "; Game ID: " + str(self.gameid)
        for i in self.odds:
            temp = temp + "\nBook: " + i[0] + "|Price 1: " + str(i[2]) + "; Price 2: " + str(i[3]) + "| Last Updated: " + str(i[1])
        return temp
# g1 = Game(2,"hi","hi")
# g2 = Game(3,"hi","hi")
# g1.addodd("hi","hi",2,2)
# g1.addodd("hey","hey",3,3)
# print(g1)

In [156]:
games = json.loads(api_response.text)

Games = []

for game in games:
    tempgame = Game(game["id"],game["bookmakers"][0]["markets"][0]["outcomes"][0]["name"],game["bookmakers"][0]["markets"][0]["outcomes"][1]["name"])
    for bookmaker in game["bookmakers"]:
        if(bookmaker["markets"][0]["key"] == "h2h"):
            tempgame.addodd(bookmaker["key"], bookmaker["last_update"],float(bookmaker["markets"][0]["outcomes"][0]["price"]), float(bookmaker["markets"][0]["outcomes"][1]["price"]))
    Games.append(tempgame)
    

Game: Seattle Seahawks vs. Tampa Bay Buccaneers; Game ID: e8dce84fd844264b94713f2ce384fbcb
Book: fanduel|Price 1: 2.2; Price 2: 1.7| Last Updated: 2022-11-12T07:56:33Z
Book: betonlineag|Price 1: 2.27; Price 2: 1.68| Last Updated: 2022-11-12T08:00:37Z
Book: twinspires|Price 1: 2.23; Price 2: 1.68| Last Updated: 2022-11-12T07:57:38Z
Book: sugarhouse|Price 1: 2.26; Price 2: 1.7| Last Updated: 2022-11-12T08:00:15Z
Book: barstool|Price 1: 2.23; Price 2: 1.68| Last Updated: 2022-11-12T08:00:37Z
Book: superbook|Price 1: 2.3; Price 2: 1.67| Last Updated: 2022-11-12T08:00:10Z
Book: betrivers|Price 1: 2.26; Price 2: 1.7| Last Updated: 2022-11-12T07:55:56Z
Book: draftkings|Price 1: 2.2; Price 2: 1.71| Last Updated: 2022-11-12T08:00:15Z
Book: lowvig|Price 1: 2.27; Price 2: 1.68| Last Updated: 2022-11-12T07:58:57Z
Book: williamhill_us|Price 1: 2.22; Price 2: 1.69| Last Updated: 2022-11-12T07:59:11Z
Book: unibet_us|Price 1: 2.23; Price 2: 1.68| Last Updated: 2022-11-12T07:55:45Z
Book: pointsbetus|Pr

In [160]:
# Arbitrage % = ((1 / decimal odds for outcome A) x 100) + ((1 / decimal odds for outcome B) x 100)

# Rafael Nadal win: (1 / 1.18) x 100 = 84.746%

# Kyle Edmund win: (1 / 7.00) x 100 = 14.286%

# 84.75% + 14.29% = 99.032% (less than 100%, therefore an arbitrage bet)

def checkArb(odd1: float, odd2: float, stake: float)->Union[list, None]:
    total: float = (1/odd1) + (1/odd2)
    if(total< 1):
        stake1: float = (stake * (1/odd1)) / total
        stake2: float = (stake * (1/odd2)) / total
        profit: float = (stake1 * odd1) - stake
        return [stake1, stake2, profit]
    return None

# print(checkArb(1.18, 7.0, 500))


[427.87286063569684, 72.12713936430318, 4.889975550122244]


In [172]:
def checkGame(game: Game):
    maxodd1 = ["No arb opportunity was found", 0, 0, 0]
    maxodd2 = ["No arb opportunity was found", 0, 0, 0]
    for book in game.odds:
        if(book[2] > maxodd1[2]):
            maxodd1 = book.copy()
        elif(book[3] > maxodd2[3]):
            maxodd2 = book.copy()
    result = checkArb(maxodd1[2],maxodd2[3],STAKE)
    if (result == None):
        print("No Arb Opportunity was found for the " + game.t1 + " v. " + game.t2 + " game. \n")
    else:
        print("Opportunity: With a total stake of $" + str(STAKE) + ", bet $" + str(result[0]) + " on the " + game.t1 + " using " + maxodd1[0] + " and $" + str(result[1])+ " on the " + game.t2 + " using " + maxodd2[0] + " for a GUARANTEED profit of $" + str(result[2]) + " (" + str(((result[2]/STAKE) * 100) + 100)+ "%)!\n")




In [173]:
for game in Games:
    checkGame(game)

No Arb Opportunity was found for the Seattle Seahawks v. Tampa Bay Buccaneers game. 

Opportunity: With a total stake of $500.0, bet $304.96453900709224 on the Buffalo Bills using betmgm and $195.03546099290782 on the Minnesota Vikings using betfair for a GUARANTEED profit of $3.1914893617021676 (100.63829787234043%)!

No Arb Opportunity was found for the Chicago Bears v. Detroit Lions game. 

No Arb Opportunity was found for the Cleveland Browns v. Miami Dolphins game. 

No Arb Opportunity was found for the Denver Broncos v. Tennessee Titans game. 

No Arb Opportunity was found for the Houston Texans v. New York Giants game. 

No Arb Opportunity was found for the Jacksonville Jaguars v. Kansas City Chiefs game. 

No Arb Opportunity was found for the New Orleans Saints v. Pittsburgh Steelers game. 

No Arb Opportunity was found for the Indianapolis Colts v. Las Vegas Raiders game. 

No Arb Opportunity was found for the Arizona Cardinals v. Los Angeles Rams game. 

No Arb Opportunity wa